## Exercise: Fill out Preprocess Step


In [1]:
%%bash
pip install stepfunctions

In [2]:
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from stepfunctions.steps.sagemaker import ProcessingStep
import sagemaker

role = get_execution_role()

PREPROCESSING_JOB_NAME = 'test-job6-preprocess'
input_data = 's3://udacity-sagemaker-solutiondata2021/l3e1/reviews_Musical_Instruments_5.json.zip'
input_preprocessing_code = 's3://udacity-sagemaker-solutiondata2021/l3e3/HelloBlazePreprocess.py'
sess = sagemaker.Session()

sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.m5.large',
                                     instance_count=1)


processed_data_train = "{}{}/{}".format("s3://", sess.default_bucket(), 'hello_blaze_train_scikit')
processed_data_test = "{}{}/{}".format("s3://", sess.default_bucket(), 'hello_blaze_test_scikit')

inputs=[ProcessingInput(source=input_data, destination='/opt/ml/processing/input', input_name = 'input-1'),  ProcessingInput(source=input_preprocessing_code , destination='/opt/ml/processing/input/code', input_name = 'code')]


outputs=[ProcessingOutput(source='/opt/ml/processing/output/train', destination=processed_data_train, output_name = 'train_data'), ProcessingOutput(source='/opt/ml/processing/output/test', destination=processed_data_test, output_name = 'test_data')]


processing_step = ProcessingStep(
    "SageMaker pre-processing step 4",
    processor=sklearn_processor,
    job_name=PREPROCESSING_JOB_NAME,
    inputs=inputs,
    outputs=outputs,
    container_entrypoint=["python3", "/opt/ml/processing/input/code/HelloBlazePreprocess.py"],
)



print(" ".join([processed_data_train, processed_data_test]))
print(inputs)


Unable to load JumpStart region config.
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/sagemaker/jumpstart/constants.py", line 69, in _load_region_config
    with open(filepath) as f:
         ^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/sagemaker/jumpstart/region_config.json'


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/stepfunctions/__init__.py:16: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


s3://sagemaker-us-east-1-698775791571/hello_blaze_train_scikit s3://sagemaker-us-east-1-698775791571/hello_blaze_test_scikit
[<sagemaker.processing.ProcessingInput object at 0x7f6227c0b140>, <sagemaker.processing.ProcessingInput object at 0x7f6228f695b0>]


## Exercise: Fill out Training Step

In [3]:
from stepfunctions.steps.sagemaker import TrainingStep
import boto3

WORKFLOW_OUTPUT = "s3://udacity-sagemaker-solutiondata2021/l3e3/workflow_output"
TRAINING_JOB_NAME = "test-job2-train"

region_name = boto3.Session().region_name
container = sagemaker.image_uris.retrieve(
    region=region_name, framework="blazingtext", version="latest"
)

helloBlazeEstimator = sagemaker.estimator.Estimator(
    container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    volume_size=30,
    max_run=360000,
    input_mode="File",
    output_path=WORKFLOW_OUTPUT,
    sagemaker_session=sess,
)

helloBlazeEstimator.set_hyperparameters(mode='supervised')

training_step = TrainingStep(
    "SageMaker Training Step",
    estimator=helloBlazeEstimator,
    data={"train": sagemaker.TrainingInput(processed_data_train, content_type="text/plain"), "validation": sagemaker.TrainingInput(processed_data_test, content_type="text/plain")},
    job_name=TRAINING_JOB_NAME,
    wait_for_completion=True,
)

## Exercise: Create & Execute Workflow

In [4]:
from stepfunctions.steps import Chain
from stepfunctions.workflow import Workflow

workflow_role = 'arn:aws:iam::565094796913:role/step-function-execution-role'

workflow_graph = Chain([processing_step, training_step])
workflow = Workflow(
    name="SageMakerProcessingWorkflow7",
    definition=workflow_graph,
    role=workflow_role,
)

workflow.create()

# Execute workflow
execution = workflow.execute(
    inputs={
        "PreprocessingJobName": PREPROCESSING_JOB_NAME,  # Each pre processing job (SageMaker processing job) requires a unique name,
        "TrainingJobName": TRAINING_JOB_NAME  # Each Sagemaker Training job requires a unique name,       
    }
)

execution_output = execution.get_output(wait=True)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 13>:13                                                                            │
│                                                                                                  │
│   10 │   role=workflow_role,                                                                     │
│   11 )                                                                                           │
│   12                                                                                             │
│ ❱ 13 workflow.create()                                                                           │
│   14                                                                                             │
│   15 # Execute workflow                                                                          │
│   16 execution = workflow.execute(                                                               │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/stepfunctions/workflow/stepfu │
│ nctions.py:205 in create                                                                         │
│                                                                                                  │
│   202 │   │   │   return self.state_machine_arn                                                  │
│   203 │   │                                                                                      │
│   204 │   │   try:                                                                               │
│ ❱ 205 │   │   │   self.state_machine_arn = self._create()                                        │
│   206 │   │   except self.client.exceptions.StateMachineAlreadyExists as e:                      │
│   207 │   │   │   self.state_machine_arn = self._extract_state_machine_arn(e)                    │
│   208 │   │   │   logger.error("A workflow with the same name already exists on AWS Step Funct   │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/stepfunctions/workflow/stepfu │
│ nctions.py:213 in _create                                                                        │
│                                                                                                  │
│   210 │   │   return self.state_machine_arn                                                      │
│   211 │                                                                                          │
│   212 │   def _create(self):                                                                     │
│ ❱ 213 │   │   response = self.client.create_state_machine(                                       │
│   214 │   │   │   name=self.name,                                                                │
│   215 │   │   │   definition=self.definition.to_json(pretty=self.format_json),                   │
│   216 │   │   │   roleArn=self.role,                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:602 in     │
│ _api_call                                                                                        │
│                                                                                                  │
│    599 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    600 │   │   │   │   )                                                                         │
│    601 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  602 │   │   │   return self._make_api_call(operation_nam

In [ ]:
execution.render_progress()